<a href="https://colab.research.google.com/gist/hotstaff/335dc9027cda0b28d3f9ff047b561a3c/waifu2x-chainer-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This notebook based 
# https://github.com/hirios/Waifu2x-Colab/blob/master/2xcolab.ipynb.
# Thanks share usuful script.
# Write by Hideto Manjo github/hotstaff
import os
from google.colab import files
from google.colab import drive
!git clone https://github.com/tsurumeso/waifu2x-chainer

try:
  drive.mount('/content/drive/')
except:
  exit()
  
FFMPEG_LOG_OPTION = "-loglevel error"

def setting():
    %cp '/content/drive/My Drive/'{INPUT_MOVIE} /content/
    print(f"Input file name is: {INPUT_MOVIE}")
    print(f"Output file name is {OUTPUT_MOVIE}")
    print(f"Scale: {SCALE}, Noise Reduction: {NOISE}")
    if DEBUG:
      FFMPEG_LOG_OPTION = ""

def get_fps():
    global FPS
    fps = str(os.popen(f'ffmpeg -i "/content/{INPUT_MOVIE}" 2> fps.txt ; egrep -i "fps" fps.txt').read())
    FPS = fps.split(",")[4].split()[0]
    !rm "fps.txt"
    
    if FPS is None:
      print("Error exit, ffmpeg can't get fps.")
      exit()


def get_frames():
    %rm /content/frames -rf
    %mkdir -p /content/frames

    print("Extracting frames from your video ...")
    !ffmpeg -y -i /content/{INPUT_MOVIE} /content/frames/%d.png {FFMPEG_LOG_OPTION}


def upscale_frames():
    print("\nUpscale each image ...")
    %cd /content/waifu2x-chainer
    %rm /content/upframes -rf
    %mkdir -p /content/upframes

    debug_option = " > /dev/null 2>&1"
    if DEBUG:
        debug_option = ""

    tta_option = ""
    if TTA:
        tta_option = "--tta"

    !python waifu2x.py -m noise_scale -n {NOISE} -s {SCALE}\
                       -i /content/frames\
                       --output /content/upframes\
                       --arch {ARCH} -g 0 {tta_option} {debug_option}


def generating_video2x():
    %cd /content/upframes
    print("\nGenerating new video file ...")

    if SOUND:
        !ffmpeg -y -i /content/{INPUT_MOVIE} -ab 192k /content/audiotrack.mp3\
                {FFMPEG_LOG_OPTION}
        !ffmpeg -y -f image2 -r {FPS} -i %d.png\
                -i /content/audiotrack.mp3 -acodec copy {OUTPUT_MOVIE}\
                {FFMPEG_LOG_OPTION}
    else:
        !ffmpeg -y -f image2 -r {FPS} -i %d.png {OUTPUT_MOVIE}\
                {FFMPEG_LOG_OPTION}


def send_for_drive():
    print("\nTransferring to drive ...")
    %cp /content/upframes/{OUTPUT_MOVIE} '/content/drive/My Drive'

In [0]:
#@title Waifu2x-chainer Upscale/Denoise Settings

#@markdown **Note this notebook required GPU runtime, so please activate GPU.**

#@markdown ---
#@markdown ### Requirements
#@markdown Please input original movie filename in your google drive "~/My Drive/".
INPUT_MOVIE =  "kakei.mp4" #@param {type:"string"}
SCALE = 1 #@param {type:"slider", min:1, max:2, step:1}
NOISE = 3 #@param {type:"slider", min:0, max:3, step:1}

#@markdown ---
#@markdown ### Optionals
PREFIX = 'waifu_' #@param ["waifu_"] {allow-input: true}
SOUND = True #@param {type:"boolean"}
ARCH = "VGG7" #@param ["VGG7", "UpConv7", "ResNet10", "UpResNet10"]
TTA = True #@param {type: "boolean"}
DEBUG = False #@param {type:"boolean"}

OUTPUT_MOVIE = f"{PREFIX}{INPUT_MOVIE[0:-4]}.mp4"
FPS = None

if __name__ == "__main__":
    setting()
    get_fps()
    get_frames()
    upscale_frames()
    generating_video2x()
    send_for_drive()